# Vehicular Collisions in Seattle

## Setup

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import bokeh
import matplotlib.pyplot as plt

df = pd.read_csv("seattle_collisions_cleaned.csv")

## Adding New Columns

### Time Columns 
We added new time columns for us to use to explore our data
***Specifically: month, year, week_of_year, day_of_year, quarter, month_name, time, floored_time, hour, hours of daylight, night hours***

In [2]:
# converting the incident date column to date time format
df["collision_incident_date"] = pd.to_datetime(df['collision_incident_date'])

# adding new time columns for us to use to explore our data
# specifically: month, year, week_of_year, day_of_year, quarter, month_name, time, floored_time, hour, hours of daylight

df["month"] = pd.to_datetime(df['collision_incident_date']).dt.month
df["year"] = pd.to_datetime(df['collision_incident_date']).dt.year
df["weekofyear"] = pd.to_datetime(df['collision_incident_date']).dt.weekofyear
df["dayofyear"] = pd.to_datetime(df['collision_incident_date']).dt.dayofyear
df["quarter"] = pd.to_datetime(df['collision_incident_date']).dt.quarter
df["month_name"] = pd.to_datetime(df['collision_incident_date']).dt.month_name()

df["time"] = pd.to_datetime(df['collision_incident_time']).dt.time
df["floored_time"] = pd.to_datetime(df['collision_incident_time']).dt.floor("H").dt.time
df["hour"] = pd.to_datetime(df['collision_incident_time']).dt.hour

# Compute hours of daylight
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - pd.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

df['daylight_hrs'] = list(map(hours_of_daylight, df.collision_incident_date))
df["night_hrs"] = 24 - df["daylight_hrs"]

In [24]:
df["quarter_name"] = pd.to_datetime(df['collision_incident_date']).dt.quarter_name()

AttributeError: 'DatetimeProperties' object has no attribute 'quarter_name'

### Adding Weather Columns
Sourced from NOAA Weather

**Added:** 

- Average Wind Speed
- Precipitation
- Temp Max 
- Temp Min

In [8]:
weather = pd.read_csv("weather.csv")

In [13]:
weather["collision_incident_date"] = pd.to_datetime(weather['DATE'])
df_weather = pd.merge(df, weather, how = "inner", on = "collision_incident_date")

In [21]:
df_weather = df_weather.rename(index=str, columns={'DATE':'date_weather', 'AWND':'average_wind_speed', 'PRCP':'precipation', 'TMAX':'temp_max', 'TMIN': 'temp_min'})
df_weather.columns

Index(['Unnamed: 0', 'collision_long', 'collision_lat',
       'collision_incident_key', 'collision_report_no', 'collision_status',
       'collision_address_type', 'collision_intersection_key',
       'collision_address', 'collision_severity_code',
       'collision_severity_description', 'collision_type',
       'collision_person_count', 'collision_pedestrian_count',
       'collision_bicycle_count', 'collision_vehicle_count',
       'collision_total_injuries', 'collision_total_serious_injuries',
       'collision_fatalities', 'collision_incident_date',
       'collision_incident_time', 'collision_junction_type',
       'collision_sdot_col_code', 'collision_sdot_col_desc',
       'collision_due_to_inattention',
       'collision_under_influence_of_drugs_alcohol',
       'collision_weather_conditions', 'collision_road_conditions',
       'collision_light_conditions', 'collision_pedestrian_right_of_way',
       'collision_sdot_colnum', 'collision_speeding_factor_in_collision',
       '

## Missingness in our Data

In [3]:
missing_values_count = df.isnull().sum()
missing_values_count

Unnamed: 0                                         0
collision_long                                  5216
collision_lat                                   5216
collision_incident_key                             0
collision_report_no                                0
collision_status                                   0
collision_address_type                          3588
collision_intersection_key                    137759
collision_address                               4392
collision_severity_code                            1
collision_severity_description                     0
collision_type                                 23113
collision_person_count                             0
collision_pedestrian_count                         0
collision_bicycle_count                            0
collision_vehicle_count                            0
collision_total_injuries                           0
collision_total_serious_injuries                   0
collision_fatalities                          

In [ ]:
fig, ax = plt.subplots(figsize=(50,420))
sns.heatmap(df.isnull().reset_index(drop=True),ax=ax, cbar = False, yticklabels = 50)

plt.ylabel("Row number", size = 22)
plt.xlabel("Feature name", size = 22)
plt.title("Missing Data", size = 32)

## Summary of our Data

Key Insights:

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,203190.0,1.015955e+05,5.865604e+04,1.000000e+00,5.079825e+04,1.015955e+05,1.523928e+05,2.031900e+05
collision_long,197974.0,-1.223309e+02,2.995239e-02,-1.224191e+02,-1.223489e+02,-1.223308e+02,-1.223121e+02,-1.222390e+02
collision_lat,197974.0,4.762041e+01,5.562823e-02,4.749557e+01,4.757862e+01,4.761614e+01,4.766399e+01,4.773414e+01
collision_incident_key,203190.0,1.288449e+05,7.475677e+04,1.001000e+03,6.708325e+04,1.180595e+05,1.843188e+05,3.064440e+05
collision_intersection_key,65431.0,3.727568e+04,5.039210e+04,2.380700e+04,2.864300e+04,2.996500e+04,3.395900e+04,6.739740e+05
collision_person_count,203190.0,2.230641e+00,1.483843e+00,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,9.300000e+01
collision_pedestrian_count,203190.0,3.712289e-02,1.989319e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00
collision_bicycle_count,203190.0,2.692554e-02,1.633493e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
collision_vehicle_count,203190.0,1.742507e+00,8.190322e-01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.500000e+01
collision_total_injuries,203190.0,3.742359e-01,7.367922e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.800000e+01


In [14]:
df_weather.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,203189.0,1.015960e+05,5.865576e+04,1.000000e+00,5.079900e+04,1.015960e+05,1.523930e+05,2.031900e+05
collision_long,197973.0,-1.223309e+02,2.995246e-02,-1.224191e+02,-1.223489e+02,-1.223308e+02,-1.223121e+02,-1.222390e+02
collision_lat,197973.0,4.762041e+01,5.562836e-02,4.749557e+01,4.757862e+01,4.761614e+01,4.766399e+01,4.773414e+01
collision_incident_key,203189.0,1.288455e+05,7.475643e+04,1.001000e+03,6.708400e+04,1.180600e+05,1.843190e+05,3.064440e+05
collision_intersection_key,65430.0,3.727580e+04,5.039248e+04,2.380700e+04,2.864300e+04,2.996500e+04,3.395900e+04,6.739740e+05
collision_person_count,203189.0,2.230652e+00,1.483838e+00,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,9.300000e+01
collision_pedestrian_count,203189.0,3.712307e-02,1.989324e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00
collision_bicycle_count,203189.0,2.692567e-02,1.633497e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
collision_vehicle_count,203189.0,1.742516e+00,8.190251e-01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.500000e+01
collision_total_injuries,203189.0,3.742378e-01,7.367935e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.800000e+01


In [5]:
df.dtypes

Unnamed: 0                                             int64
collision_long                                       float64
collision_lat                                        float64
collision_incident_key                                 int64
collision_report_no                                   object
collision_status                                      object
collision_address_type                                object
collision_intersection_key                           float64
collision_address                                     object
collision_severity_code                               object
collision_severity_description                        object
collision_type                                        object
collision_person_count                                 int64
collision_pedestrian_count                             int64
collision_bicycle_count                                int64
collision_vehicle_count                                int64
collision_total_injuries

In [23]:
df_weather.to_csv("seattle_collisions_weather.csv")

In [25]:
df_weather.describe()

,Unnamed: 0,collision_long,collision_lat,collision_incident_key,collision_intersection_key,collision_person_count,collision_pedestrian_count,collision_bicycle_count,collision_vehicle_count,collision_total_injuries,...,weekofyear,dayofyear,quarter,hour,daylight_hrs,night_hrs,average_wind_speed,precipation,temp_max,temp_min
count,203189.000000,197973.000000,197973.000000,203189.000000,65430.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,...,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000,203189.000000
mean,101595.998898,-122.330854,47.620409,128845.503457,37275.795430,2.230652,0.037123,0.026926,1.742516,0.374238,...,26.783286,184.241612,2.524777,10.054998,12.055346,11.944654,7.583058,0.131224,60.939539,46.208510
std,58655.758168,0.029952,0.055628,74756.433229,50392.481354,1.483838,0.198932,0.163350,0.819025,0.736794,...,14.871170,104.070674,1.108897,7.517786,2.597868,2.597868,3.124137,0.279816,13.011373,8.900393
min,1.000000,-122.419091,47.495573,1001.000000,23807.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,8.218894,8.218905,0.000000,0.000000,25.000000,14.000000
25%,50799.000000,-122.348898,47.578622,67084.000000,28643.000000,2.000000,0.000000,0.000000,2.000000,0.000000,...,14.000000,96.000000,2.000000,0.000000,9.552073,9.401723,5.370000,0.000000,51.000000,40.000000
50%,101596.000000,-122.330842,47.616140,118060.000000,29965.000000,2.000000,0.000000,0.000000,2.000000,0.000000,...,27.000000,185.000000,3.000000,11.000000,12.069940,11.930060,6.930000,0.000000,59.000000,47.000000
75%,152393.000000,-122.312095,47.663995,184319.000000,33959.000000,3.000000,0.000000,0.000000,2.000000,1.000000,...,40.000000,275.000000,4.000000,16.000000,14.598277,14.447927,9.400000,0.140000,70.000000,53.000000
max,203190.000000,-122.238951,47.734142,306444.000000,673974.000000,93.000000,6.000000,2.000000,15.000000,78.000000,...,53.000000,366.000000,4.000000,23.000000,15.781095,15.781106,21.470000,3.770000,103.000000,71.000000
